<a href="https://colab.research.google.com/github/zhangxs131/Text-Generation/blob/main/CodeGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CodeGen

看看CodeGen生成的代码怎么，最复杂能生成什么样子的，是不是比我自己做力扣还强，hh

原论文：https://arxiv.org/pdf/2203.13474v3.pdf

github项目地址 ：https://github.com/salesforce/CodeGen

官方colab demo地址 ：https://colab.research.google.com/drive/1fQI8OgzMAR0bquCrvhlAtXSw6iMFbVgI#scrollTo=wPwrvvMlkTVA

In [1]:
#安装依赖包
!git clone https://github.com/salesforce/CodeGen
%cd CodeGen
!pip install --upgrade pip setuptools
!pip install -r requirements.txt

Cloning into 'CodeGen'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 104 (delta 47), reused 63 (delta 18), pack-reused 0
Receiving objects: 100% (104/104), 1.32 MiB | 23.26 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/CodeGen
     |████████████████████████████████| 2.1 MB 8.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly=

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 28.0 MB/s eta 0:00:01tcmalloc: large alloc 2041348096 bytes == 0x55c81b5fc000 @  0x7f91f6f131e7 0x55c817db3407 0x55c817d7d17c 0x55c817e5d47a 0x55c817d7ff9d 0x55c817e71d4d 0x55c817df3ec8 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817d817aa 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817df0719 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 30.4 MB/s eta 0:00:01tcmalloc: large alloc 2551685120 bytes == 0x55c8950c4000 @  0x7f91f6f14615 0x55c817d7d17c 0x55c817e5d47a 0x55c817d7ff9d 0x55c817e71d4d 0x55c817df3ec8 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817deea2e 0x55c817d8188a 0x55c817defb4f 0x55c817de

#加载model和tokenizer

这里使用的模型为
codegen-350M-mono

350M为模型大小

mono为data类型
*   nl是在英语文本预训练的模型本身没有生成code能力
*   mutli是在nl模型基础上，使用多种编程语言语料下进行训练
*   mono 是初始化在multi上在python编程文件下的语料进行训练

In [4]:
chosen_model="codegen-350M-mono" #可选择参数有 ["codegen-350M-nl", "codegen-350M-multi", "codegen-350M-mono", "codegen-2B-nl", "codegen-2B-multi", "codegen-2B-mono", "codegen-6B-nl", "codegen-6B-multi", "codegen-6B-mono", "codegen-16B-nl", "codegen-16B-multi", "codegen-16B-mono"]
fp16=True 

import os
!mkdir checkpoints
#下载模型
if not os.path.exists(f'./checkpoints/{chosen_model}'):
  !wget -P checkpoints https://storage.googleapis.com/sfr-codegen-research/checkpoints/{chosen_model}.tar.gz && tar -xvf checkpoints/{chosen_model}.tar.gz -C checkpoints/

import torch
from jaxformer.hf.sample import truncate as do_truncate
from jaxformer.hf.sample import set_env,set_seed,print_time, create_model ,create_custom_gpt2_tokenizer,create_tokenizer,sample


models_nl = ['codegen-350M-nl', 'codegen-2B-nl', 'codegen-6B-nl', 'codegen-16B-nl']
models_pl = ['codegen-350M-multi', 'codegen-2B-multi', 'codegen-6B-multi', 'codegen-16B-multi', 'codegen-350M-mono', 'codegen-2B-mono', 'codegen-6B-mono', 'codegen-16B-mono']
models = models_nl + models_pl

set_env()

pad = 50256
device = torch.device('cuda:0')
ckpt = f'./checkpoints/{chosen_model}'

with print_time('loading parameters'):
  model = create_model(ckpt=ckpt, fp16=fp16).to(device)


with print_time('loading tokenizer'):
  if chosen_model in models_pl:
    tokenizer = create_custom_gpt2_tokenizer()
  else:
    tokenizer = create_tokenizer()
  tokenizer.padding_side = 'left'
  tokenizer.pad_token = pad


mkdir: cannot create directory ‘checkpoints’: File exists
--2022-04-17 01:39:40--  https://storage.googleapis.com/sfr-codegen-research/checkpoints/codegen-350M-mono.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 656148604 (626M) [application/x-tar]
Saving to: ‘checkpoints/codegen-350M-mono.tar.gz’

codegen-350M-mono.t 100%[===================>] 625.75M   210MB/s    in 3.0s    

2022-04-17 01:39:43 (210 MB/s) - ‘checkpoints/codegen-350M-mono.tar.gz’ saved [656148604/656148604]

codegen-350M-mono/
codegen-350M-mono/config.json
codegen-350M-mono/pytorch_model.bin
loading parameters
loading parameters took 24.49s
loading tokenizer


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading tokenizer took 4.72s


#尝试CodeGen生成代码

In [34]:
def codegen(context):
  rng_seed=1143
  rng_deterministric=True
  p=0.90 #解码使用top_P策略，其中p值
  t=0.2
  max_length=128
  batch_size=1
  #context="def sum of two numbers(a,b):"

  with print_time('sampling'):
    completion = sample(device=device, model=model, tokenizer=tokenizer, context=context, pad_token_id=pad, num_return_sequences=batch_size, temp=t, top_p=p, max_length_sample=max_length)[0]
    truncation = do_truncate(completion)

    # print('=' * 100)
    # print(completion)
    print('=' * 100)
    print(context+truncation)
    print('=' * 100)

In [35]:
codegen('def helloworld():')
#看着还行

sampling
def helloworld():
    print("Hello World")

helloworld()


sampling took 0.70s


In [44]:
codegen('all the prime number less than 100')
#感觉有点奇怪，其实有一次真的做出来了这倒题

sampling
all the prime number less than 100
    """
    for i in range(2, 100):
        prime = True
        for j in range(2, i):
            if i % j == 0:
                prime = False
                break
        if prime:
            print(i)

def main():
    """
    Main function
    """
    prime_number_check()
    prime_number_check_2()
    prime_number_check_3()
    prime_number_check_4()
    prime_number_check_5()
    prime_number_check
sampling took 5.48s


In [45]:
codegen('def gen_all_prime_number_less_than(number):')
#看着还可以了，就是没有给出is_prime函数

sampling
def gen_all_prime_number_less_than(number):
    prime_numbers = []
    for i in range(2, number):
        if is_prime(i):
            prime_numbers.append(i)
    return prime_numbers

def gen_all_prime_number_greater_than(number):
    prime_numbers = []
    for i in range(2, number):
        if is_prime(i):
            prime_numbers.append(i)
    return prime_numbers


sampling took 5.48s


In [46]:
codegen('def bert_model():')

sampling
def bert_model():
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.nn import CrossEntropyLoss, MSELoss
    from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
    from torch.nn.utils.rnn import pad_sequence
    from torch.nn.utils.rnn import pack_padded_sequence, pack_padded_sequence_left
    from torch.nn.utils.rnn import pad_packed_sequence
    from torch.nn
sampling took 5.55s


In [47]:
codegen('class Transformer:')

sampling
class Transformer:
    def __init__(self, model_name, model_path, device, max_len=512,
                 num_layers=6, num_heads=8, d_model=512, d_k=64, d_v=64,
                 dropout=0.1, max_len=512, num_layers=6, num_heads=8,
                 d_model=512, d_k=64, d_v=64,
                 use_cuda=True,
                 use_cuda_cuda=False,

sampling took 6.64s


In [49]:
codegen('class MNIST:')

sampling
class MNIST:
    def __init__(self, path):
        self.path = path
        self.train_data = self.load_data()
        self.test_data = self.load_data()
        self.train_labels = self.load_labels()
        self.test_labels = self.load_labels()
        self.train_images = self.load_images()
        self.test_images = self.load_images()
        self.train_images = self.train_images.reshape(60000, 28,
sampling took 5.55s


In [50]:
codegen('CNN Network')

sampling
CNN Network
        self.model = self.create_model()
        self.model.load_weights(self.model_path)

    def create_model(self):
        """
        Creates a new model instance.
        """
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu',
                         input_shape=(150, 150, 3)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(32, (3, 3),
sampling took 5.55s


In [53]:
#leetcode最长回文字串
codegen('class Solution: \
    def longestPalindrome(self, s: str) -> str:')
#可能是长度限制，不过效果还行

sampling
class Solution:     def longestPalindrome(self, s: str) -> str:
        if len(s) == 1:
            return s
        if len(s) == 2:
            return s[0] * 2

        dp = [[0 for _ in range(len(s))] for _ in range(len(s))]
        for i in range(len(s)):
            for j in range(i, len(s)):
                if s[i] == s[j]:
                    dp[i][j] = dp[i + 1][j - 1] + 2
                else:
                    dp[i
sampling took 5.62s


#命令行生成代码



In [54]:
!python -m jaxformer.hf.sample --model codegen-350M-mono --context "def hello_world():"

loading parameters
loading parameters took 11.39s
loading tokenizer
loading tokenizer took 2.38s
sampling

    print("Hello World")

hello_world()

#
def hello_world():
    print("Hello World")

hello_world()


sampling took 0.65s
done.
